In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras import optimizers
import pandas as pd
from keras.layers.normalization import BatchNormalization

/Users/meetkumarpatel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
classifier = Sequential()

classifier.add(Conv2D(64, kernel_size=(3, 3), strides=(1,1), input_shape=(64,64,3), name='Input_Conv2D_1', activation='relu'))
classifier.add(BatchNormalization(axis=-1, name='BatchNorm_1'))
classifier.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool_1'))

classifier.add(Conv2D(64, kernel_size=(3, 3), strides=(1,1), activation='relu', name='Conv2D_2'))
classifier.add(BatchNormalization(axis=-1, name='BatchNorm_2'))
classifier.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool_2'))
classifier.add(Dropout(0.25, name='Dropout'))

classifier.add(Conv2D(64, kernel_size=(3, 3), strides=(1,1), name='Conv2D_3', activation='relu'))
classifier.add(BatchNormalization(axis=-1, name='BatchNorm_3'))
classifier.add(MaxPooling2D(pool_size=(2,2), name='MaxPool_3'))

classifier.add(Conv2D(32, kernel_size=(3, 3), strides=(1,1), name='Conv2D_4', activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2), name='MaxPool_4'))

classifier.add(Flatten(name='Flatten')) # Fully connected layer
classifier.add(BatchNormalization(name='BatchNorm_Flatten'))

classifier.add(Dense(128, name='Dense_1', activation='relu'))
classifier.add(Dense(64, name='Dense_2', activation='relu'))

classifier.add(Dense(1, activation='sigmoid', name='Sigmoid_Output'))
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_Conv2D_1 (Conv2D)      (None, 62, 62, 64)        1792      
_________________________________________________________________
BatchNorm_1 (BatchNormalizat (None, 62, 62, 64)        256       
_________________________________________________________________
MaxPool_1 (MaxPooling2D)     (None, 31, 31, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 29, 29, 64)        36928     
_________________________________________________________________
BatchNorm_2 (BatchNormalizat (None, 29, 29, 64)        256       
_________________________________________________________________
MaxPool_2 (MaxPooling2D)     (None, 14, 14, 64)        0         
_________________________________________________________________
Dropout (Dropout)            (None, 14, 14, 64)        0         
__________

In [17]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipvalue=0.5)
classifier.compile( optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
from keras.preprocessing.image import ImageDataGenerator
# applying transformation to image
train_datagen = ImageDataGenerator(
 rescale=1./255,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
training_set = train_datagen.flow_from_directory(
        '/users/meetkumarpatel/desktop/face_model/dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')
test_set = test_datagen.flow_from_directory(
        '/users/meetkumarpatel/desktop/face_model/dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 15804 images belonging to 2 classes.
Found 7044 images belonging to 2 classes.


In [ ]:
from IPython.display import display 

history = classifier.fit_generator(
                training_set,
                steps_per_epoch=300,
                epochs=15,
                validation_data=test_set, validation_steps=150)

In [21]:
classifier.save("face_1.h5",overwrite=True)
#classifier.summary()


In [10]:
training_set.class_indices

{'no': 0, 'yes': 1}

# test

In [13]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img("/users/meetkumarpatel/desktop/face_test_no.jpg",target_size=(64, 64))
# Adding the channel
test_image = image.img_to_array(test_image)
# adding the batch size as predict method expects
test_image = np.expand_dims(test_image, axis=0)
# Predicting the test image
result= classifier.predict(test_image)
print(result)

[[0.]]


In [14]:
print(result[0][0])

0.0
